# Neural Particles 2D

## 2D Dataset
Pipeline:
* Generation of high-res data (reference) using some random cubes of water
* Extracting relevant data (particle data and grid data) like e.g. sdf, velocity, pressure, density...
* Down-sampling of particles (by a given factor) and generation of the low-res data (source)
* Extract corresponding and relevant patches on the surface from the data-set pairs (considering the low-res data)
* Use patche-pairs to train the NN

### Setup

Simple structs containing parameters for different data-set and neural network versions.

Data-set Parameters:
* **prefix**: prefix of filename
* **nn_version**: used neural network version
* **patch_size**: size of generated surface patches (of the low-res data)
* **stride**: stride used for the generation of the patches
* **surf**: surface tolerance, specifies which sdf-values are considered as surface (for patches)
* **fps**: frames per second, the velocity of the simulation
* **frame_count**: count of generated frames
* **sub_res**: count of particles per cell (per dimension)
* **res**: resolution of high-res grid
* **factor**: goal up-scale factor of particles
* **upres**: specify if the low-res grids will be up-scaled to res (=> input and output same scale)
* **seed**: seed for random data-set generation
* **min_scale**: minimum scale of boxes for data-set generation
* **max_scale**: maximum scale of boxes for data-set generation
* **min_pos**: minimum x-position of boxes
* **max_pos**: maximum x-position of boxes
* **max_h**: maximum start-height of boxes
* **max_cnt**: maximum count of boxes

Neural Network Parameters:
* **train_data_count**: count of training data-sets
* **val_data_count**: count of validation data-sets
* **test_data_count**: count of test data-sets
* **batch_size**: batch size used for training and validation
* **learning_rate**: learing rate used for training
* **epochs**: training epochs

In [310]:
class DatasetParam:
    def __init__(self, prefix="sph_2D", nn_version=0, patch_size=9, stride=3, surf=0.2, 
                fps=30, frame_count=50, sub_res=2, res=150, factor=10, upres=1,
                seed=124820112, min_scale=0.05, max_scale=0.3,
                min_pos=0.2,max_pos=0.8,max_h=0.1,max_cnt=5):
    
        self.prefix = prefix
        self.nn_version = nn_version
        self.patch_size = patch_size
        self.stride = stride
        self.surf = surf

        self.fps = fps
        self.frame_count = frame_count
        self.sub_res = sub_res
        self.res = res
        self.factor = factor
        self.upres = upres

        self.seed = seed
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.min_pos = min_pos
        self.max_pos = max_pos
        self.max_h = max_h
        self.max_cnt = max_cnt    
        
class NNParam:
    def __init__(self, train_data_count=5, val_data_count=1, test_data_count=0,
                batch_size=32,learning_rate=0.002,epochs=1000):
        self.train_data_count = train_data_count
        self.val_data_count = val_data_count
        self.test_data_count = test_data_count
        self.batch_size = batch_size
        self.learning_rate=learning_rate
        self.epochs = epochs


Data Version:
* **V0**: default values, input and output patches same size
* **V1**: smaller patches, input and output patches different size

NN Version:
* **V0**: CNN for patches with same size
* **V1**: CNN for patches with different size

In [320]:
data_version_param = [
    DatasetParam(),
    DatasetParam(upres=0,
                 factor=9,
                 stride=2,
                 surf=0.5,
                 patch_size=5,
                 nn_version=1)
]

nn_version_param = [
    NNParam(),
    NNParam()
]

In [321]:
import os
from subprocess import Popen, PIPE

import datetime

manta_loc = "2D_SPH/"

data_version = 1
data_param = data_version_param[data_version]

nn_param = nn_version_param[data_param.nn_version]

data_loc = "2D_data/"
src = "lowres"
ref = "highres"

# count of training/validation setups
data_count = nn_param.train_data_count + nn_param.val_data_count + nn_param.test_data_count

verbose = False

def createFolder(p):
    if not os.path.exists(p):
        os.makedirs(p)
    
def create_curr_date_folder(path):
    now = datetime.datetime.now()
    path += "%04d%02d%02d"%(now.year,now.month,now.day)
    createFolder(path)
    return path + "/"

createFolder(data_loc)
createFolder(data_loc+src)
createFolder(data_loc+ref)
createFolder(data_loc+"patches")
createFolder(data_loc+"patches/"+src)
createFolder(data_loc+"patches/"+ref)
createFolder(data_loc+"result")
createFolder(data_loc+"screenshots")

def remove_data(path_prefix):
    command = ["rm", path_prefix+"*"]
    print(" ".join(command) + "\n")
    proc = Popen(command, stdin=None, stdout=PIPE, stderr=PIPE)
    if verbose:
        for line in proc.stdout:
            print(line.decode('utf-8'))
    for line in proc.stderr:
        print(line.decode('utf-8'))

def run_manta(scene, param={}):
    command = [manta_loc+"build/manta", manta_loc+scene]

    for k, v in param.items():
        command += [k, str(v)]
        
    print(" ".join(command) + "\n")

    proc = Popen(command, stdin=None, stdout=PIPE, stderr=PIPE)

    if verbose:
        for line in proc.stdout:
            print(line.decode('utf-8'))
    for line in proc.stderr:
        print(line.decode('utf-8'))

Compute 2D down-scale factor:
$$lowres=\frac{res}{\sqrt(factor)}$$

In [315]:
import math

#remove_data(src_prefix)

low_res = data_param.res
high_patch_size = data_param.patch_size
factor_2D = data_param.factor
if not data_param.upres:
    factor_2D = math.sqrt(data_param.factor)
    low_res = int(low_res/factor_2D)
    high_patch_size = int(high_patch_size*factor_2D)
    
print("2D factor: %f" % factor_2D)
print("low resolution: %d" % low_res)
print("big patch size. %d" % high_patch_size)

2D factor: 3.000000
low resolution: 50
big patch size. 15


### Generate High-res Data

In [316]:
import random

ref_prefix = "%s%s/ref_%s_v%02d" % (data_loc, ref, data_param.prefix, data_version)

#remove_data(ref_prefix)

param = {}

#disable gui
param['gui'] = 0

# resolution of domain
param['res'] = data_param.res
param['sres'] = data_param.sub_res

# output file format
param['out'] = ref_prefix

# write only every 30th frame -> 30 frames are one timestep
param['fps'] = data_param.fps

# simulation time (how many frames)
param['t_end'] = float(data_param.frame_count) / data_param.fps

# run random training setups
random.seed(data_param.seed)
for i in range(data_count):
    param['out'] = (ref_prefix + "_d%03d")%i + "_%03d"
    
    # generate different cubes with dataformat "pos_x,pos_y,scale_x,scale_y"
    param['c_cnt'] = random.randint(1,data_param.max_cnt)
    cubes = {}
    for c in range(param['c_cnt']):    
        scx = random.uniform(data_param.min_scale, data_param.max_scale)
        scy = random.uniform(data_param.min_scale, data_param.max_scale)
        px = random.uniform(data_param.min_pos+scx/2, data_param.max_pos-scx/2)
        py = random.uniform(0, data_param.max_h) + scy/2
        print("%f,%f,%f,%f"%(px,py,scx,scy))
        cubes['c%d'%c] = "%f,%f,%f,%f"%(px,py,scx,scy)
    run_manta("scenes/2D_sph.py", dict(param, **cubes))

0.323035,0.134318,0.145915,0.178643
0.547369,0.164307,0.131005,0.299544
0.513754,0.070534,0.077867,0.098838
0.667608,0.116032,0.062978,0.062918
2D_SPH/build/manta 2D_SPH/scenes/2D_sph.py c0 0.323035,0.134318,0.145915,0.178643 c1 0.547369,0.164307,0.131005,0.299544 c2 0.513754,0.070534,0.077867,0.098838 t_end 1.6666666666666667 c3 0.667608,0.116032,0.062978,0.062918 sres 2 res 150 fps 30 out 2D_data/highres/ref_sph_2D_v01_d000_%03d gui 0 c_cnt 4

dyld: Library not loaded: @rpath/QtOpenGL.framework/Versions/5/QtOpenGL

  Referenced from: /Users/Lukas/Neural Particles/2D_SPH/build/manta

  Reason: image not found

0.343429,0.153053,0.220075,0.153738
2D_SPH/build/manta 2D_SPH/scenes/2D_sph.py c0 0.343429,0.153053,0.220075,0.153738 t_end 1.6666666666666667 sres 2 res 150 fps 30 out 2D_data/highres/ref_sph_2D_v01_d001_%03d gui 0 c_cnt 1

dyld: Library not loaded: @rpath/QtOpenGL.framework/Versions/5/QtOpenGL

  Referenced from: /Users/Lukas/Neural Particles/2D_SPH/build/manta

  Reason: imag

### Generate Low-res Data

In [167]:
src_prefix = "%s%s/%s_v%02d" % (data_loc, src, data_param.prefix, data_version)

param = {}

param['res'] = data_param.res
param['sres'] = data_param.sub_res
param['upres'] = data_param.upres
 
param['factor'] = data_param.factor
param['gui'] = 0
param['t'] = data_param.frame_count

for i in range(data_count):
    param['in'] = (ref_prefix + "_d%03d")%i + "_%03d"
    param['out'] = (src_prefix + "_d%03d")%i + "_%03d"
    run_manta("scenes/down_scale.py", param)

2D_SPH/build/manta 2D_SPH/scenes/down_scale.py upres 0 sres 2 out 2D_data/lowres/sph_2D_v01_d000_%03d factor 9 in 2D_data/highres/ref_sph_2D_v01_d000_%03d res 150 gui 0 t 50

2D_SPH/build/manta 2D_SPH/scenes/down_scale.py upres 0 sres 2 out 2D_data/lowres/sph_2D_v01_d001_%03d factor 9 in 2D_data/highres/ref_sph_2D_v01_d001_%03d res 150 gui 0 t 50

2D_SPH/build/manta 2D_SPH/scenes/down_scale.py upres 0 sres 2 out 2D_data/lowres/sph_2D_v01_d002_%03d factor 9 in 2D_data/highres/ref_sph_2D_v01_d002_%03d res 150 gui 0 t 50

Traceback (most recent call last):

  File "2D_SPH/scenes/down_scale.py", line 90, in <module>

    pp.load(path + "_ps.uni")

RuntimeError: can't open file 2D_data/highres/ref_sph_2D_v01_d002_031_ps.uni

Error raised in /Users/Lukas/Neural Particles/2D_SPH/source/fileio.cpp:1156

2D_SPH/build/manta 2D_SPH/scenes/down_scale.py upres 0 sres 2 out 2D_data/lowres/sph_2D_v01_d003_%03d factor 9 in 2D_data/highres/ref_sph_2D_v01_d003_%03d res 150 gui 0 t 50

2D_SPH/build/manta

### Extract Surface-Patches

In [323]:
src_patches_path = "%spatches/%s/%s_v%02d" % (data_loc, src, data_param.prefix, data_version)
ref_patches_path = "%spatches/%s/ref_%s_v%02d" % (data_loc, ref, data_param.prefix, data_version)

#remove_data(src_patches_path)
#remove_data(ref_patches_path)

param = {}

param["t"] = data_param.frame_count

# patch size
param["psize"] = data_param.patch_size
param["stride"] = data_param.stride

param["hpsize"] = high_patch_size

# tolerance of surface
param["surface"] = data_param.surf

for i in range(data_count):
    param["h_in"] = (ref_prefix + "_d%03d")%i + "_%03d"
    param["l_in"] = (src_prefix + "_d%03d")%i + "_%03d"
    param["h_out"] = (ref_patches_path + "_d%03d")%i + "_%03d"
    param["l_out"] = (src_patches_path + "_d%03d")%i + "_%03d"
    run_manta("scenes/extract_patches.py", param)

2D_SPH/build/manta 2D_SPH/scenes/extract_patches.py l_in 2D_data/lowres/sph_2D_v01_d000_%03d hpsize 15 l_out 2D_data/patches/lowres/sph_2D_v01_d000_%03d h_out 2D_data/patches/highres/ref_sph_2D_v01_d000_%03d stride 2 t 50 psize 5 h_in 2D_data/highres/ref_sph_2D_v01_d000_%03d surface 0.5

2D_SPH/build/manta 2D_SPH/scenes/extract_patches.py l_in 2D_data/lowres/sph_2D_v01_d001_%03d hpsize 15 l_out 2D_data/patches/lowres/sph_2D_v01_d001_%03d h_out 2D_data/patches/highres/ref_sph_2D_v01_d001_%03d stride 2 t 50 psize 5 h_in 2D_data/highres/ref_sph_2D_v01_d001_%03d surface 0.5

2D_SPH/build/manta 2D_SPH/scenes/extract_patches.py l_in 2D_data/lowres/sph_2D_v01_d002_%03d hpsize 15 l_out 2D_data/patches/lowres/sph_2D_v01_d002_%03d h_out 2D_data/patches/highres/ref_sph_2D_v01_d002_%03d stride 2 t 50 psize 5 h_in 2D_data/highres/ref_sph_2D_v01_d002_%03d surface 0.5

Traceback (most recent call last):

  File "2D_SPH/scenes/extract_patches.py", line 41, in <module>

    header, l_data = readUni(pat

### Show Data

In [169]:
param = {}
dataset = 0

# show low res
param['in'] = (src_prefix + "_d%03d")%dataset + "_%03d_ps.uni"
param['sdf'] = (src_prefix + "_d%03d")%dataset + "_%03d_sdf.uni"
param['t'] = data_param.frame_count
param['res'] = low_res

param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + "sph_%03d_sdf.png"

run_manta("scenes/show_particles.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_particles.py sdf 2D_data/lowres/sph_2D_v01_d000_%03d_sdf.uni t 50 in 2D_data/lowres/sph_2D_v01_d000_%03d_ps.uni res 50 scr 2D_data/screenshots/20171102/sph_%03d_sdf.png

QThread: Destroyed while thread is still running

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMethod: No such method QWidget::screenshot(QString)

QMetaObject::invokeMet

In [170]:
param = {}
dataset = 0

# show high res
param['in'] = (ref_prefix + "_d%03d")%dataset + "_%03d_ps.uni"
param['sdf'] = (ref_prefix + "_d%03d")%dataset + "_%03d_sdf.uni"
param['t'] = data_param.frame_count
param['res'] = data_param.res

param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + "sph_%03d_sdf_ref.png"

run_manta("scenes/show_particles.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_particles.py sdf 2D_data/highres/ref_sph_2D_v01_d000_%03d_sdf.uni t 50 in 2D_data/highres/ref_sph_2D_v01_d000_%03d_ps.uni res 150 scr 2D_data/screenshots/20171102/sph_%03d_sdf_ref.png

QThread: Destroyed while thread is still running



In [324]:
param = {}
dataset = 0
timestep = 10

# show patches
param['src'] = (src_patches_path + "_d%03d")%dataset + "_%03d_sdf"%timestep
param['ref'] = (ref_patches_path + "_d%03d")%dataset + "_%03d_sdf"%timestep
param['psize'] = data_param.patch_size
param['hpsize'] = high_patch_size
param['t'] = 1#data_param.frame_count

#param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + "sph_patch_%03d_sdf_ref.png"

run_manta("scenes/show_patches.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_patches.py ref 2D_data/patches/highres/ref_sph_2D_v01_d000_010_sdf src 2D_data/patches/lowres/sph_2D_v01_d000_010_sdf hpsize 15 psize 5 t 1

QThread: Destroyed while thread is still running



## Train Neural Network

### Setup Neural Network

In [325]:
import sys
sys.path.append("2D_SPH/scenes/tools")
from dataset import Dataset

timestep = 20

src_patches_path = "%spatches/%s/%s_v%02d" % (data_loc, src, data_param.prefix, data_version) + "_d%03d" + "_%03d_sdf" % timestep
print(src_patches_path)
ref_patches_path = "%spatches/%s/ref_%s_v%02d" % (data_loc, ref, data_param.prefix, data_version) + "_d%03d" + "_%03d_sdf" % timestep
print(ref_patches_path)

train_data = Dataset(src_patches_path, ref_patches_path, 0, nn_param.train_data_count)
val_data = Dataset(src_patches_path, ref_patches_path, nn_param.train_data_count, nn_param.train_data_count + nn_param.val_data_count)

2D_data/patches/lowres/sph_2D_v01_d%03d_020_sdf
2D_data/patches/highres/ref_sph_2D_v01_d%03d_020_sdf


In [326]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

input_shape = (data_param.patch_size, data_param.patch_size, 1)
output_shape = (high_patch_size, high_patch_size, 1)

model = Sequential()

model.add(keras.layers.Conv2D(filters=3, kernel_size=3, 
                              strides=1, input_shape=input_shape, 
                              activation='tanh', padding='same'))

model.add(keras.layers.Conv2D(filters=3, kernel_size=3,
                              strides=1, activation='tanh', padding='same'))
#model.add( keras.layers.BatchNormalization() )  

model.add(Dense(units=10))
model.add(Activation('tanh'))
#model.add( keras.layers.Dropout(0.25) )  

if data_param.nn_version == 1:
    model.add(keras.layers.Conv2DTranspose(filters=1, kernel_size=3, 
                                           strides=3, activation='tanh', padding='same'))
    
else:
    model.add(keras.layers.Conv2DTranspose(filters=1, kernel_size=3, 
                                           strides=1, activation='tanh', padding='same'))
    

model.add(keras.layers.Conv2DTranspose(filters=1, kernel_size=3, 
                                       strides=1, activation='tanh', padding='same'))

model.compile( loss='mse', optimizer=keras.optimizers.adam(lr=nn_param.learning_rate) )

#x,y = train_data.get_batch(1000)
model.fit(x=train_data.data,y=train_data.ref_data, epochs=nn_param.epochs, batch_size=nn_param.batch_size)

Epoch 1/1000
520/520 [==============================] - 0s - loss: 1.3613     
Epoch 2/1000
520/520 [==============================] - 0s - loss: 1.0949     
Epoch 3/1000
520/520 [==============================] - 0s - loss: 0.9226     
Epoch 4/1000
520/520 [==============================] - 0s - loss: 0.8788     
Epoch 5/1000
520/520 [==============================] - 0s - loss: 0.8536     
Epoch 6/1000
520/520 [==============================] - 0s - loss: 0.8366     
Epoch 7/1000
520/520 [==============================] - 0s - loss: 0.8267     
Epoch 8/1000
520/520 [==============================] - 0s - loss: 0.8182     
Epoch 9/1000
520/520 [==============================] - 0s - loss: 0.8121     
Epoch 10/1000
520/520 [==============================] - 0s - loss: 0.8079     
Epoch 11/1000
520/520 [==============================] - 0s - loss: 0.8028     
Epoch 12/1000
520/520 [==============================] - 0s - loss: 0.8000     
Epoch 13/1000
520/520 [==========================

In [327]:
loss_and_metrics = model.evaluate(x=val_data.data, y=val_data.ref_data, batch_size=nn_param.batch_size)
print(loss_and_metrics)

 32/146 [=====>........................] - ETA: 0s0.630016420802


### Run Neural Network

In [328]:
from uniio import *

test_filename = "%sresult/%s_v%02d" % (data_loc, data_param.prefix, data_version) + "_%03d_sdf" % timestep
ref_filename = "%sresult/ref_%s_v%02d" % (data_loc, data_param.prefix, data_version) + "_%03d_sdf" % timestep
result_filename = "%sresult/result_%s_v%02d" % (data_loc, data_param.prefix, data_version) + "_%03d_sdf" % timestep

#remove_data(test_filename)
#remove_data(ref_filename)
#remove_data(result_filename)

result = model.predict(x=val_data.data, batch_size=nn_param.batch_size)

for patch in val_data.data:
    writeNumpyBuf(test_filename, patch)
    
for patch in val_data.ref_data:
    writeNumpyBuf(ref_filename, patch)
    
for patch in result:
    writeNumpyBuf(result_filename, patch)
    
finalizeNumpyBufs()

### Show Results

In [329]:
param = {}

# show patches
param['src'] = test_filename
param['ref'] = result_filename
param['ref2'] = ref_filename
param['psize'] = data_param.patch_size
param['hpsize'] = high_patch_size
param['t'] = 1

param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + "sph_patch_%03d_sdf_res.png"

run_manta("scenes/show_patches.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_patches.py hpsize 15 psize 5 scr 2D_data/screenshots/20171102/sph_patch_%03d_sdf_res.png ref 2D_data/result/result_sph_2D_v01_020_sdf src 2D_data/result/sph_2D_v01_020_sdf ref2 2D_data/result/ref_sph_2D_v01_020_sdf t 1

QThread: Destroyed while thread is still running



In [335]:
result = np.ndarray(shape=(data_param.res,data_param.res,1), dtype=float)
weights = np.ndarray(shape=(data_param.res,data_param.res,1), dtype=int)
result.fill(0)
weights.fill(0)

border = 2
result=np.pad(result,((border,border),(border,border),(0,0)),mode="edge")
weights=np.pad(weights,((border,border),(border,border),(0,0)),mode="edge")
print(result.shape)
dataset = nn_param.train_data_count

input_path = src_prefix + "_d%03d_%03d_sdf.uni" % (dataset, timestep)
ref_input_path = ref_prefix + "_d%03d_%03d_sdf.uni" % (dataset, timestep)
print(input_path)
output_path = "%s/%s_v%02d_d%03d_%03d_result.uni" % (data_loc, data_param.prefix, data_version, dataset, timestep)

hdr, source = readUni(input_path)

ps = data_param.patch_size//2
hps = high_patch_size//2

for x in range (ps, low_res-ps, 1):
    for y in range(ps, low_res-ps, 1):
        if(abs(source[0,x,y,0]) < data_param.surf):
            x0=x-ps
            x1=x+ps+1
            y0=y-ps
            y1=y+ps+1
            predict = model.predict(x=np.array([source[0,x0:x1,y0:y1]]), batch_size=1)

            x0=int(factor_2D*x)-hps+border
            x1=int(factor_2D*x)+hps+border+1
            y0=int(factor_2D*y)-hps+border
            y1=int(factor_2D*y)+hps+border+1
                
            result[x0:x1,y0:y1,0] += predict[0,:,:,0]
            weights[x0:x1,y0:y1,0] += 1
            
for x in range(border,data_param.res+border):
    for y in range(border,data_param.res+border):
        if weights[x,y,0] > 0:
            result[x,y,0] /= weights[x,y,0]

hdr['dimX'] = data_param.res
hdr['dimY'] = data_param.res

#print(result[border:data_param.res+border,border:data_param.res+border,0].shape)
writeUni(output_path, hdr, result[border:data_param.res+border,border:data_param.res+border,0])

(154, 154, 1)
2D_data/lowres/sph_2D_v01_d005_020_sdf.uni


In [340]:
param = {}
dataset = 0

scr_path = "%s_v%02d_d%03d_%03d" % (data_param.prefix, data_version, dataset, timestep)

# show input
param['sdf'] = input_path
param['t'] = 1
param['res'] = low_res

param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + scr_path + "_src.png"

run_manta("scenes/show_particles.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_particles.py sdf 2D_data/lowres/sph_2D_v01_d005_020_sdf.uni t 1 res 50 scr 2D_data/screenshots/20171102/sph_2D_v01_d000_020_src.png



In [341]:
param = {}
dataset = 0

# show result
param['sdf'] = output_path
param['t'] = 1
param['res'] = data_param.res

param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + scr_path + "_res.png"

run_manta("scenes/show_particles.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_particles.py sdf 2D_data//sph_2D_v01_d005_020_result.uni t 1 res 150 scr 2D_data/screenshots/20171102/sph_2D_v01_d000_020_res.png



In [342]:
param = {}
dataset = 0

# show result
param['sdf'] = ref_input_path
param['t'] = 1
param['res'] = data_param.res

param['scr'] = create_curr_date_folder(data_loc+'screenshots/') + scr_path + "_ref.png"

run_manta("scenes/show_particles.py", param)

2D_SPH/build/manta 2D_SPH/scenes/show_particles.py sdf 2D_data/highres/ref_sph_2D_v01_d005_020_sdf.uni t 1 res 150 scr 2D_data/screenshots/20171102/sph_2D_v01_d000_020_ref.png

